## Testing

### Generating Gif Model Predictions

In [25]:
import torch
import collections
import pandas as pd
import torchvision.transforms as transforms
from torchvision.models import vit_b_16
from PIL import Image
import imageio
import torch.nn.functional as F

In [17]:
# Load GIF metadata file
CSV_FILE_PATH = "/home/jecroisp/Thesis/processed_data/TestingData/gif_metadata_blip.csv"  # Use the latest file
OUTPUT_CSV_FILE = "gif_model_predictions.csv"

In [7]:
# Load the trained ViT model
MODEL_PATH = "/home/jecroisp/Thesis/processed_data/p_crema/emotion_recognition_vit.pth"  # Ensure this is the correct path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Load saved state_dict
state_dict = torch.load(MODEL_PATH, map_location="cpu")

# Fix key mismatch by removing ".0" from "heads.0.weight"
new_state_dict = collections.OrderedDict()
for key, value in state_dict.items():
    new_key = key.replace("heads.0.", "heads.")  # Adjust key names
    new_state_dict[new_key] = value

/tmp/ipykernel_40639/1490301435.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location="cpu")


In [11]:
class EmotionRecognitionViT(torch.nn.Module):
    def __init__(self, num_classes=6):
        super(EmotionRecognitionViT, self).__init__()
        self.vit = vit_b_16(weights=None)  # Load model architecture
        self.vit.heads = torch.nn.Linear(self.vit.heads[0].in_features, num_classes)

    def forward(self, x):
        return self.vit(x)

# Initialize model
model = EmotionRecognitionViT(num_classes=6)

# Load fixed state_dict
model.load_state_dict(new_state_dict)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set to evaluation mode

print("Model loaded successfully!")

Model loaded successfully!


In [22]:

# Define the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize frames
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

def preprocess_gif(gif_path):
    """Extracts the middle frame from a GIF, converts it to RGB, and applies transformations."""
    gif = imageio.mimread(gif_path)  # Read all frames
    middle_frame = gif[len(gif) // 2]  # Get the middle frame
    frame_image = Image.fromarray(middle_frame).convert("RGB")  # Ensure 3 channels (RGB)
    
    tensor_image = transform(frame_image)  # Apply transformations
    tensor_image = tensor_image.unsqueeze(0)  # Add batch dimension (1, 3, 224, 224)

    return tensor_image


In [23]:


# Emotion labels (same order as training)
emotion_labels = ["Anger", "Disgust", "Fear", "Happiness", "Neutral", "Sadness"]

def predict_emotion(gif_path):
    """Predicts emotion confidence scores for a GIF using the trained model."""
    input_tensor = preprocess_gif(gif_path).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)  # Forward pass
        probabilities = F.softmax(outputs, dim=1).cpu().numpy().flatten()  # Convert logits to probabilities

        # Get final predicted label and its confidence score
        predicted_idx = torch.argmax(outputs, 1).item()
        predicted_label = emotion_labels[predicted_idx]
        predicted_confidence = probabilities[predicted_idx]

    # Return all probabilities + final verdict
    return {
        "File_Name": gif_path,
        "Predicted_Emotion": predicted_label,
        "Confidence_Score": predicted_confidence,
        "Anger_Score": probabilities[0],
        "Disgust_Score": probabilities[1],
        "Fear_Score": probabilities[2],
        "Happiness_Score": probabilities[3],
        "Neutral_Score": probabilities[4],
        "Sadness_Score": probabilities[5],
    }



In [26]:

df = pd.read_csv(CSV_FILE_PATH)

# Process each GIF and predict its emotion
predictions = []

for index, row in df.iterrows():
    gif_path = row["File_Name"]
    result = predict_emotion(gif_path)  # Get predictions
    predictions.append(result)

    print(f"Processed: {gif_path} → {result['Predicted_Emotion']} ({result['Confidence_Score']:.2f})")

# Convert results to DataFrame
df_results = pd.DataFrame(predictions)

# Save results to CSV
df_results.to_csv(OUTPUT_CSV_FILE, index=False)

print(f"Emotion prediction completed! Data saved to: {OUTPUT_CSV_FILE}")

Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/2XskdWPH2WxnxfMnZAI.gif → Neutral (0.54)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/S5Pgd5fykea99hAm8r.gif → Sadness (0.50)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/PJTQJPEoWvuKY.gif → Neutral (0.84)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/ysoK6qc1xpNxm.gif → Neutral (0.79)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/asLOZFUdcYsfK.gif → Happiness (0.49)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/1310XHyc6yC6ju.gif → Anger (0.42)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/k90nbKSCPUfra.gif → Neutral (0.61)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/jPJz9s6VAqLLi.gif → Neutral (0.85)
Processed: /home/jecroisp/Thesis/processed_data/TestingData/downLoadedGifs/qUrOyBLlXGwXS.gif → Fear (0.72)
P

### Accuracy Calculation

In [6]:
pip install ace

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import ace as tools

In [2]:
# Paths to CSV files
MODEL_PREDICTIONS_FILE = "gif_model_predictions.csv"  # Output from the model
SENTIMENT_DATA_FILE = "/home/jecroisp/Thesis/processed_data/TestingData/gif_metadata_sentiment.csv"  # Ground truth labels
HUGGING_FACE_LABEL= "/home/jecroisp/Thesis/processed_data/TestingData/gif_sentiment_mapped.csv"

# Load both datasets
df_predictions = pd.read_csv(MODEL_PREDICTIONS_FILE)
df_sentiment = pd.read_csv(SENTIMENT_DATA_FILE)
df_hugging = pd.read_csv(HUGGING_FACE_LABEL)
df_hugging.drop(columns= "GIF_ID" )
df_hugging.drop(columns= "Caption" )
df_hugging.drop(columns= "Enhanced_Caption" )
# Merge data on the "File_Name" column to align predictions with ground truth labels
df_merge = df_predictions.merge(df_sentiment, on="File_Name")
df_merged = df_merge.merge(df_hugging, on="File_Name")
df_merged.head(5)

,File_Name,Predicted_Emotion,Confidence_Score_x,Anger_Score,Disgust_Score,Fear_Score,Happiness_Score,Neutral_Score,Sadness_Score,GIF_ID_x,Caption_x,Enhanced_Caption_x,VADER_Emotion,BERT_Emotion,GIF_ID_y,Caption_y,Enhanced_Caption_y,GO_Emotion,Confidence_Score_y,Mapped_Emotion
0,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.544303,0.029619,0.077486,0.068970,0.207776,0.544303,0.071846,2XskdWPH2WxnxfMnZAI,Happy Hour Drinking GIF,a woman in a dress sitting at a table,Neutral,Happiness,2XskdWPH2WxnxfMnZAI,Happy Hour Drinking GIF,a woman in a dress sitting at a table,curiosity,0.103617,Neutral
1,/home/jecroisp/Thesis/processed_data/TestingDa...,Sadness,0.500611,0.067243,0.015973,0.083644,0.004043,0.328486,0.500611,S5Pgd5fykea99hAm8r,Michael J Fox Film GIF by BAFTA,a man in a suit and glasses is standing at a p...,Neutral,Happiness,S5Pgd5fykea99hAm8r,Michael J Fox Film GIF by BAFTA,a man in a suit and glasses is standing at a p...,excitement,0.098083,Happiness
2,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.839233,0.004797,0.010715,0.021950,0.122772,0.839233,0.000533,PJTQJPEoWvuKY,Cruel Intentions Parallels GIF,a woman in a dress standing in front of a crowd,Neutral,Happiness,PJTQJPEoWvuKY,Cruel Intentions Parallels GIF,a woman in a dress standing in front of a crowd,excitement,0.119565,Happiness
3,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.792544,0.063218,0.030381,0.055864,0.006521,0.792544,0.051472,ysoK6qc1xpNxm,Scared Who Framed Roger Rabbit GIF,a cartoon character in a red dress and a white...,Neutral,Sadness,ysoK6qc1xpNxm,Scared Who Framed Roger Rabbit GIF,a cartoon character in a red dress and a white...,amusement,0.276729,Happiness
4,/home/jecroisp/Thesis/processed_data/TestingDa...,Happiness,0.491529,0.004162,0.033166,0.001866,0.491529,0.467237,0.002039,asLOZFUdcYsfK,Howard The Duck 80S GIF,a white cat with a pink shirt,Neutral,Sadness,asLOZFUdcYsfK,Howard The Duck 80S GIF,a white cat with a pink shirt,amusement,0.094472,Happiness


In [3]:
df_predictions.head(5)

,File_Name,Predicted_Emotion,Confidence_Score,Anger_Score,Disgust_Score,Fear_Score,Happiness_Score,Neutral_Score,Sadness_Score
0,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.544303,0.029619,0.077486,0.068970,0.207776,0.544303,0.071846
1,/home/jecroisp/Thesis/processed_data/TestingDa...,Sadness,0.500611,0.067243,0.015973,0.083644,0.004043,0.328486,0.500611
2,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.839233,0.004797,0.010715,0.021950,0.122772,0.839233,0.000533
3,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.792544,0.063218,0.030381,0.055864,0.006521,0.792544,0.051472
4,/home/jecroisp/Thesis/processed_data/TestingDa...,Happiness,0.491529,0.004162,0.033166,0.001866,0.491529,0.467237,0.002039


In [4]:
df_merged.drop(columns ="Caption_y")
df_merged.drop(columns ="Enhanced_Caption_y")
df_merged.drop(columns ="GIF_ID_y")
df_merged.rename(columns = {"Confidence_Score_y":"Confidence_Go"})
df_merged.head(5)

,File_Name,Predicted_Emotion,Confidence_Score_x,Anger_Score,Disgust_Score,Fear_Score,Happiness_Score,Neutral_Score,Sadness_Score,GIF_ID_x,Caption_x,Enhanced_Caption_x,VADER_Emotion,BERT_Emotion,GIF_ID_y,Caption_y,Enhanced_Caption_y,GO_Emotion,Confidence_Score_y,Mapped_Emotion
0,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.544303,0.029619,0.077486,0.068970,0.207776,0.544303,0.071846,2XskdWPH2WxnxfMnZAI,Happy Hour Drinking GIF,a woman in a dress sitting at a table,Neutral,Happiness,2XskdWPH2WxnxfMnZAI,Happy Hour Drinking GIF,a woman in a dress sitting at a table,curiosity,0.103617,Neutral
1,/home/jecroisp/Thesis/processed_data/TestingDa...,Sadness,0.500611,0.067243,0.015973,0.083644,0.004043,0.328486,0.500611,S5Pgd5fykea99hAm8r,Michael J Fox Film GIF by BAFTA,a man in a suit and glasses is standing at a p...,Neutral,Happiness,S5Pgd5fykea99hAm8r,Michael J Fox Film GIF by BAFTA,a man in a suit and glasses is standing at a p...,excitement,0.098083,Happiness
2,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.839233,0.004797,0.010715,0.021950,0.122772,0.839233,0.000533,PJTQJPEoWvuKY,Cruel Intentions Parallels GIF,a woman in a dress standing in front of a crowd,Neutral,Happiness,PJTQJPEoWvuKY,Cruel Intentions Parallels GIF,a woman in a dress standing in front of a crowd,excitement,0.119565,Happiness
3,/home/jecroisp/Thesis/processed_data/TestingDa...,Neutral,0.792544,0.063218,0.030381,0.055864,0.006521,0.792544,0.051472,ysoK6qc1xpNxm,Scared Who Framed Roger Rabbit GIF,a cartoon character in a red dress and a white...,Neutral,Sadness,ysoK6qc1xpNxm,Scared Who Framed Roger Rabbit GIF,a cartoon character in a red dress and a white...,amusement,0.276729,Happiness
4,/home/jecroisp/Thesis/processed_data/TestingDa...,Happiness,0.491529,0.004162,0.033166,0.001866,0.491529,0.467237,0.002039,asLOZFUdcYsfK,Howard The Duck 80S GIF,a white cat with a pink shirt,Neutral,Sadness,asLOZFUdcYsfK,Howard The Duck 80S GIF,a white cat with a pink shirt,amusement,0.094472,Happiness


In [5]:
# Check matches for VADER sentiment
df_merged["Correct_VADER"] = df_merged["Predicted_Emotion"] == df_merged["VADER_Emotion"]

# Check matches for BERT sentiment
df_merged["Correct_BERT"] = df_merged["Predicted_Emotion"] == df_merged["BERT_Emotion"]

#check matches for hugging face sentiment
df_merged["Correct_GO"] = df_merged["Predicted_Emotion"] == df_merged["Mapped_Emotion"]


# Calculate Accuracy
vader_accuracy = df_merged["Correct_VADER"].mean() * 100  # Percentage
bert_accuracy = df_merged["Correct_BERT"].mean() * 100  # Percentage
go_accuracy = df_merged["Correct_GO"].mean() * 100

print(f"Model Accuracy Compared to VADER Sentiment: {vader_accuracy:.2f}%")
print(f"Model Accuracy Compared to BERT Sentiment: {bert_accuracy:.2f}%")
print(f"Model Accuracy Compared to GO from Hugging Emotion Sentiment: {go_accuracy:.2f}%")

Model Accuracy Compared to VADER Sentiment: 69.17%
Model Accuracy Compared to BERT Sentiment: 1.50%
Model Accuracy Compared to GO from Hugging Emotion Sentiment: 54.64%
